LoRA trainer

In [14]:
#makes a metadata.csv file to create the dataset
import os
labels = {}
dataset_path = "loras/hu_tao/dataset"
labels_paths = dataset_path + '/labels'
for file in os.listdir(labels_paths):
    if file.endswith(".txt"):
        with open(os.path.join(labels_paths, file), 'r') as f:
            name = file.split(".")[0] + ".jpg"
            labels[name] = f.read()

metadata = 'file_name,text\n'
for k, v in labels.items():
    metadata += f'{k},"{v}"\n'
with open(dataset_path + "/metadata.csv", "w") as f:
    f.write(metadata)
    

In [26]:
#requires to upload the dataset to the huggingface hub
#huggingface-cli login has to be executed before running this code
from datasets import load_dataset

dataset_name = "hu-tao-v2"
dataset = load_dataset('imagefolder', data_dir='loras/hu_tao/dataset')
dataset.push_to_hub(dataset_name, private = False)

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.76s/it]


In [25]:
dataset["train"][2]['image']	
print(dataset["train"].column_names)

['image', 'text']


In [ ]:
MODEL_NAME="runwayml/stable-diffusion-v1-5"
OUTPUT_DIR="lora_train/loras/hu_tao2"
DATASET_NAME="XayEss/" + dataset_name

!accelerate launch --mixed_precision="fp16"  train_text_to_image_lora.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --dataset_name=$DATASET_NAME \
  --dataloader_num_workers=0 \
  --resolution=512 \
  --center_crop \
  --random_flip \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --max_train_steps=15000 \
  --learning_rate=1e-04 \
  --max_grad_norm=1 \
  --lr_scheduler="cosine" \
  --lr_warmup_steps=0 \
  --output_dir=${OUTPUT_DIR} \
  --checkpointing_steps=1000 \
  --validation_prompt="1 girl, hu tao, genshin impact, forest, spear" \
  --seed=1337 \
  --validation_epochs=100 \
  --rank=20

['image', 'text']


12/13/2023 23:02:00 - INFO - __main__ - Distributed environment: DistributedType.NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

{'thresholding', 'timestep_spacing', 'prediction_type', 'sample_max_value', 'clip_sample_range', 'dynamic_thresholding_ratio', 'variance_type'} was not found in config. Values will be initialized to default values.
{'scaling_factor', 'force_upcast'} was not found in config. Values will be initialized to default values.
{'use_linear_projection', 'time_embedding_type', 'time_embedding_dim', 'dual_cross_attention', 'cross_attention_norm', 'time_cond_proj_dim', 'class_embed_type', 'conv_in_kernel', 'encoder_hid_dim_type', 'reverse_transformer_layers_per_block', 'addition_embed_type_num_heads', 'num_attention_heads', 'addition_time_embed_dim', 'resnet_out_scale_factor', 'mid_block_type', 'encoder_hid_dim', 'resnet_time_scale_shift', 'transformer_layers_per_block', 'only_cross_attention', 'time_embedding_act_fn'

accelerate launch --mixed_precision="fp16"  train_text_to_image_lora.py --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" --dataset_name="XayEss/hu-tao-v2" --dataloader_num_workers=0 --resolution=512 --center_crop --random_flip --train_batch_size=2 --gradient_accumulation_steps=4 --max_train_steps=15000 --learning_rate=1e-04 --max_grad_norm=1 --lr_scheduler="cosine" --lr_warmup_steps=0 --output_dir="loras/hu_tao/lora" --checkpointing_steps=1000 --validation_prompt="1 girl, hu tao, genshin impact, forest, spear" --seed=1337 --validation_epochs=100